<a href="https://colab.research.google.com/github/raitharnett/tensorflow-great-barrier-reef/blob/main/cots_mask_r_nn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [ ]:
%%bash
mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json
python -m pip install --upgrade pip
pip install --upgrade --force-reinstall --no-deps kaggle
kaggle competitions download -c tensorflow-great-barrier-reef
unzip -d  tensorflow-great-barrier-reef tensorflow-great-barrier-reef.zip 

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
%%bash
python -m pip install --upgrade pip
git clone https://github.com/tensorflow/models.git 
git clone --depth 1 https://github.com/tensorflow/models
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

In [ ]:
%%bash
wget http://download.tensorflow.org/models/object_detection/tf2/20200711/mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8.tar.gz 
tar -xvzf mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8.tar.gz 

In [ ]:
import os
import pathlib
import matplotlib
import matplotlib.pyplot as pltls
import pandas as pd
import io
import scipy.misc
import shutil
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from six.moves.urllib.request import urlopen
import tensorflow as tf
import tensorflow_hub as hub
from object_detection.utils import dataset_util
from object_detection.dataset_tools import tf_record_creation_util
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util
from sklearn.model_selection import train_test_split
import contextlib2
from pathlib import Path
from enum import Enum

class COTSClass(Enum):
  COTS = 1
  
%matplotlib inline
tf.get_logger().setLevel('ERROR')
print(f"TF version: {tf.__version__}")

In [ ]:
# path constants
COTS_DATA_ROOT = '/content/'
COTS_DATA =  os.path.join(COTS_DATA_ROOT,'tensorflow-great-barrier-reef')
COTS_DATA_IMAGES = os.path.join(COTS_DATA,'train_images')
COTS_DATASET = '/content/dataset'
Path(COTS_DATASET).mkdir(parents=True, exist_ok=True)
COTS_DATA_TRAIN_TF_RECORDS = f'{COTS_DATASET}/train'
COTS_DATA_TEST_TF_RECORDS = f'{COTS_DATASET}/test'

In [ ]:
# load COTS data
def cots_annotations(data):
  import json
  return json.loads(data.replace("'", '"'))

def cots_image_path(row):
  return  os.path.join(COTS_DATA_IMAGES,
                       f"video_{row.video_id}",
                       f"{row.video_frame}.jpg")

cots_df = pd.read_csv(os.path.join(COTS_DATA,'train.csv'), converters={'annotations':cots_annotations})
cots_df['image_path'] = cots_df.apply(cots_image_path, axis=1)
cots_train_test_split = train_test_split(cots_df, train_size = 0.8)

In [ ]:
FORMAT = 'jpeg'.encode('utf8')
CLASS_NAME = COTSClass.COTS.name.encode('utf8')

def create_cots_tf_example(row): 
  contents = tf.io.read_file(row.image_path)
  image = tf.io.decode_jpeg(contents, channels=3)
  h,w,_ = image.shape
  filename = row.image_id.encode('utf8')
  boxes = np.array([[a['x'], a['x'] + a['width'], a['y'], a['y'] + a['height']] for a in row.annotations], dtype='float64')
  xmin, xmax, ymin, ymax, classes_text, classes = [], [], [], [], [], []
  if  (0 < boxes.size):
    # normalize
    boxes[:,[0,1]] *= 1/w
    boxes[:,[2,3]] *= 1/h
    xmin, xmax = np.transpose(boxes[:,[0,1]]).tolist()
    ymin, ymax = np.transpose(boxes[:,[2,3]]).tolist()
    classes_text = [CLASS_NAME for i in range(len(xmin))] 
    classes = [COTSClass.COTS.value for i in range(len(xmin))]
  feature = { 'image/height': dataset_util.int64_feature(h),
              'image/width': dataset_util.int64_feature(w),
              'image/filename': dataset_util.bytes_feature(filename),
              'image/source_id': dataset_util.bytes_feature(filename),
              'image/encoded': dataset_util.bytes_feature(contents.numpy()),
              'image/format': dataset_util.bytes_feature(FORMAT),
              'image/object/bbox/xmin': dataset_util.float_list_feature(xmin),
              'image/object/bbox/xmax': dataset_util.float_list_feature(xmax),
              'image/object/bbox/ymin': dataset_util.float_list_feature(ymin),
              'image/object/bbox/ymax': dataset_util.float_list_feature(ymax),
              'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
              'image/object/class/label': dataset_util.int64_list_feature(classes),
            }
  return tf.train.Example(features=tf.train.Features(feature=feature))

def load_cots(df, base_path, num_shards=10):
  with contextlib2.ExitStack() as stack:
    records = tf_record_creation_util.open_sharded_output_tfrecords(stack, base_path, num_shards)
    for index, row in df.iterrows():
      example = create_cots_tf_example(row)
      shard_index = index % num_shards
      records[shard_index].write(example.SerializeToString())

In [ ]:
cots_tf_record_keys = [COTS_DATA_TRAIN_TF_RECORDS, COTS_DATA_TEST_TF_RECORDS]
cots_train_test_data = {cots_tf_record_keys[i]: cots_train_test_split[i] for i in range(len(cots_train_test_split))}
for base_path, df in cots_train_test_data.items():
  load_cots(df, base_path)

In [ ]:
from string import Template

TRAINING_STEPS = 1000
WARMUP_STEPS = 100
BATCH_SIZE = 2

MODEL_LABEL_MAP = "/content/drive/MyDrive/cots/label_map.txt"
MODEL_PIPELINE_CONFIG_TEMPLATE = "/content/drive/MyDrive/cots/mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8.config"


COTS_MODEL_DIR='/content/drive/MyDrive/cots/mask_rcnn_inception_resnet_v2'

PIPELINE_CONFIG_PATH = '/content/dataset/pipeline.config'
LABEL_MAP_PATH = '/content/dataset/label_map.pbtxt'

shutil.copy(MODEL_LABEL_MAP, LABEL_MAP_PATH)

category_index = label_map_util.create_category_index_from_labelmap(LABEL_MAP_PATH, use_display_name=True)

with open(MODEL_PIPELINE_CONFIG_TEMPLATE, mode='r') as f:
  config_file_template = f.read()
pipeline = Template(config_file_template).substitute(training_steps=TRAINING_STEPS, warmup_steps=WARMUP_STEPS, batch_size=BATCH_SIZE)
with open(PIPELINE_CONFIG_PATH, mode='w') as f:
  f.write(pipeline)

In [ ]:
%%bash -s "$COTS_MODEL_DIR" "$PIPELINE_CONFIG_PATH"
MODEL_DIR=$1
PIPELINE_CONFIG_PATH=$2
# train model
python models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path=$PIPELINE_CONFIG_PATH \
    --model_dir=$MODEL_DIR \
    --alsologtostderr
# # evaluate model
# python models/research/object_detection/model_main_tf2.py \
#     --pipeline_config_path=$PIPELINE_CONFIG_PATH \
#     --model_dir=$MODEL_DIR \
#     --checkpoint_dir=$MODEL_DIR \
#     --eval_timeout=0 \
#     --alsologtostderr
# # save model
# python models/research/object_detection/exporter_main_v2.py \
#     --input_type image_tensor \
#     --pipeline_config_path=$PIPELINE_CONFIG_PATH \
#     --trained_checkpoint_dir=$MODEL_DIR \
#     --output_directory=$MODEL_DIR/output \
#     --alsologtostderr

In [ ]:
detector = hub.load("https://tfhub.dev/tensorflow/mask_rcnn/inception_resnet_v2_1024x1024/1")

In [ ]:
import cv2 as cv 
from google.colab.patches import cv2_imshow

COTS_DATA = '/content/drive/MyDrive/cots/tensorflow-great-barrier-reef'
COTS_DATA_IMAGES = os.path.join(COTS_DATA,'train_images')
PATH_TO_LABELS = './models/research/object_detection/data/mscoco_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)
cots_df = pd.read_csv(os.path.join(COTS_DATA,'train.csv'))
cots_df['image_path'] = cots_df.apply(lambda r: os.path.join(COTS_DATA_IMAGES,f"video_{r.video_id}",f"{r.video_frame}.jpg"), axis=1)
cots_df_elements = cots_df.sample(n=1)

for _, row in cots_df_elements.iterrows():
  image_tensor = tf.io.decode_jpeg(tf.io.read_file(row.image_path), channels=3)
  detector_output = detector(image_tensor[tf.newaxis, ...])
  result = {key:value.numpy() for key,value in detector_output.items()}
  print(result['num_detections'])
  label_id_offset = 0
  image_np =  image_tensor.numpy()
  image_np_with_mask = image_tensor.numpy().copy()
  if 'detection_masks' in result:
    # we need to convert np.arrays to tensors
    detection_masks = tf.convert_to_tensor(result['detection_masks'][0])
    detection_boxes = tf.convert_to_tensor(result['detection_boxes'][0])

    # Reframe the bbox mask to the image size.
    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
              detection_masks, detection_boxes,
                image_np.shape[0], image_np.shape[1])
    detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                        tf.uint8)
    result['detection_masks_reframed'] = detection_masks_reframed.numpy()

  viz_utils.visualize_boxes_and_labels_on_image_array(
        image_np_with_mask,
        result['detection_boxes'][0],
        (result['detection_classes'][0] + label_id_offset).astype(int),
        result['detection_scores'][0],
        category_index,
        use_normalized_coordinates=True,
        max_boxes_to_draw=200,
        min_score_thresh=.30,
        agnostic_mode=False,
        instance_masks=result.get('detection_masks_reframed', None),
        line_thickness=8)

  plt.figure(figsize=(24,32))
  plt.imshow(image_np_with_mask)
  plt.show()


